In [ ]:
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

Queries for fetching catalyst and control data

In [ ]:
q_catalyst = None
q_control = None

Build index

In [ ]:
n_friends_post_ids = {}

df_control_n_friends = q_control['n_friends'].values
df_control_post_ids = q_control['post_id'].values

for i in tqdm(range(len(df_control_post_ids))):
    
    i_post_id = df_control_post_ids[i]
    i_n_friends = df_control_n_friends[i]
    
    if np.isnan(df_control_n_friends[i]):
        continue
    
    n_friends_post_ids_l = n_friends_post_ids.get(i_n_friends, [])
    n_friends_post_ids_l.append(i_post_id)
    n_friends_post_ids[i_n_friends] = n_friends_post_ids_l

Shuffle each index

In [ ]:
for n_f in tqdm(n_friends_post_ids.keys()):    
    post_ids = n_friends_post_ids[n_f]    
    n_friends_post_ids[n_f] = random.sample(post_ids, len(post_ids))

Finding posts by users with the same number of friends

In [ ]:
def find_post(cat_n_friends, n_friends_post_ids, n_friends_max_ind, seen):
    
    post_ids = n_friends_post_ids[cat_n_friends]
    
    n_friends_max_ind[cat_n_friends] = n_friends_max_ind.get(cat_n_friends, 0)
    
    max_ind = n_friends_max_ind[cat_n_friends]
    
    if max_ind > (len(post_ids) - 1):
        return None
    
    for i in range(max_ind, len(post_ids)):
        n_friends_max_ind[cat_n_friends] += 1
        
        if post_ids[i] not in seen:
            return post_ids[i]
        
    return None

In [ ]:
max_d_friends = 5

df_catalyst_post_ids = q_catalyst['post_id'].values
df_catalyst_n_friends = q_catalyst['n_friends'].values
n_catalysts = len(df_catalyst_post_ids)

n_friends_max_ind = {}
seen = set()
matching = {} # catalyst_post_id => control_post_id

d_n_friends = []
n_not_matched = 0

for i in range(n_catalysts):
    if i % 100000 == 0: 
        print(f"#{i} | # not matched: {n_not_matched} | # seen {len(seen)}")
    
    cat_post_id = df_catalyst_post_ids[i]
    cat_n_friends = df_catalyst_n_friends[i]
            
    m_post_id = find_post(
        cat_n_friends,
        n_friends_post_ids,
        n_friends_max_ind,
        seen
    )
    
    if m_post_id is not None:
        matching[cat_post_id] = m_post_id
        seen.add(m_post_id)

    else:
        print("match not found", cat_n_friends)
        n_not_matched += 1

Output

In [ ]:
df_out = pd.DataFrame(
    data={
        "c_post_id": list(matching.keys()), 
        "m_post_id": list(matching.values())
    })